# **최적의 검색 전략 확보를 위한 테스트 파일**

1. 목적: 각 검색 전략설정이 기존에 발견된 문제를 해결하는지를 평가하고, 최적의 검색기 조합을 탐색
2. 조합 예시

    > MultiQueryRetriever + ParentDocumentRetriever

    > MultiQueryRetriever + NormalRetriever

    > ParentDocumentRetriever + SelfQueryRetrievever

3. 측정 포인트:
- 각 발생 문제가 해결하려는 검색기로 충분히 해결이 되는지 평가
- Context Recall
- 검색된 문서 수 vs 정확도


## **환경설정(실험동안 변경 X)**
- DB 연결
- 모델 설정
- 프롬프트 설정
- 임베딩 모델 설정
- 벡터스토어 설정

In [1]:
# 테스트를 위한 셋업
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 환경 변수 로드 (.env)
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Pinecone 인덱스 설정
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "hufs-chatbot"

# 모델 설정
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo"  
)

# 프롬프트 정의
prompt = PromptTemplate.from_template("""
너는 한국외대 학사 관련 질문에 답변하는 챗봇이야.
다음은 검색된 문서 내용이야. 이 내용을 바탕으로 사용자의 질문에 답변해.
답을 모르면 모른다고 해. 무조건 한국어로 높임말을 사용해서 답변해.

#질문:
{question}

#문서 내용:
{context}

#답변:
""")

# 임베딩 모델, 벡터스토어 설정
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")  

vectorstore = PineconeVectorStore.from_existing_index(
    index_name="hufs-chatbot",
    embedding=embedding,
     text_key="page_content" 
)

## **검색기 설정**
- 검색기 구조를 변경해가며 최적의 성능을 내는 검색기 확인
- 각 내용, 조합별 성능이나 문제점 등 기록 필수

In [2]:
# 실험에 따라 변경될 내용
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

## 체인
- 변경 X 순서도 retriever 설정 뒤로 오도록 할 것

In [3]:
# 체인 구성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## **실험**
- 질문은 기존에 문제를 갖고 있던 질문과 선정된 질문 들로 구성
- 답변 결과 해석: 정확히 답변 했는지, 정확한 답변이 아니라면 어떤 문제가 있었고 어떻게 개선해야 하는지

In [4]:
# 질문 - 답변 확인
question = "전과신청 시행 시기 알려줘" # 질문 입력
response = chain.invoke(question)
print(response)

2025학년도 후기이중전공 신청 기간은 4월 14일(월) 10:00부터 4월 18일(금) 16:00까지입니다.


## **검색된 문서 확인**

In [5]:
# 검색된 문서 확인
docs = vectorstore.similarity_search(question, k=5) # 검색 문서 수는 검색기에서 설정한 문서 수와 통일

for doc in docs:
    print(doc.metadata)          # 문서 메타정보 (예: source, page 등)
    print(doc.page_content)  # 문서 내용 미리보기

{'author': '학사종합지원센터', 'date': '2025.04.04', 'doc_type': '학사공지', 'ntt_id': '223498', 'title': '[공통] 후기이중전공 신청 공고', 'url': 'https://www.hufs.ac.kr/bbs/hufs/2181/223498/artclView.do'}
[공통] 후기이중전공 신청 공고

[공통] 후기이중전공 신청 공고

(공통) 후기이중전공 신청 공고

2025학년도 후기이중전공 신청 일정 및 방법 등을 공고합니다.

1. 신청기간: 2025 4. 14.(월) 10:00 ~ 4. 18.(금) 16:00까지

2. 신청대상: 현재 7~8학기 이수중인 자로, 후기이중전공을 희망하는 전공을 부전공으로 이수중인 자

(7학기를 마치고 휴학한 학생도 이번 신청기간에 신청할 것)

3. 신청방법: 인터넷 종합정보시스템에서 신청 (로그인 > 전공/교직관리 > 후기이중전공신청)

4. 허가자 발표: 7월초 홈페이지 공고(예정)

5. 기타사항

가. 1년에 1회, 4월에만 신청하므로 반드시 희망자는 이번 신청 일정을 따를 것

나. 지원학과 입학정원의 50 ~ 150% 이내에서 학과 자율 선발하되, 학과에서 요구하는 기본 자격요건 미충족 시 탈락될 수 있음

(세부 배정기준은 지원학과 학과장실 문의 또는 학과 홈페이지 참조)

다. 이수자 선발 이후 8학기까지 전공심화(+부전공) 또는 일반 부전공에 맞는 졸업에 필요한 졸업학점 요건을 모두 완료하여야 최종적으로 허가

라. 조기졸업대상자, 학석사연계과정/석사예약제 이수자는 신청 불가(신청 희망 시 내방하여 해당 자격 취소처리 후 신청)

마. 후기이중전공으로 선발되면 추가학기를 1학기 이상 무조건 등록, 수강해야 하며 등록금은 후기이중전공 학과의 등록금으로 납부해야 함

바. 기타 자세한 사항은 전자규정집 "이중전공시행규정" 참조

2025. 4.

교무처장

| 0 |
|:------------------------------|
| (공통) 후기이중전공